In [1]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd 

In [57]:
# initial variables
search_term = "Data Science"
pages = 30
records = []
col_name = ['JopTitle', 'CompName', 'CompLocation', 'PostDate', 'Description', 'JopURL']
template = 'https://th.indeed.com/งาน?q={}'
search_text = search_term.replace(' ', '+')
# add term query to url
url = template.format(search_text)
# add page query placeholder
url += '&start={}'

print(url)

https://th.indeed.com/งาน?q=Data+Science&start={}


In [ ]:
# startup the webdriver
driver = webdriver.Chrome(ChromeDriverManager().install())

for page in range(1, pages+1):
    formated_url = url.format(page)
    print(f"Scraping from url {formated_url} page {page}")
    
    driver.get(formated_url)
    driver.implicitly_wait(5) # gives an implicit wait for 5 seconds
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    ul = soup.find('ul', {'class': 'jobsearch-ResultsList'})
    results = ul.findChildren("li" , recursive=False)
    #results = soup.find_all('ul', {'class': 'jobsearch-ResultsList'})
    for item in results:
        try:
            title = item.find('a', {'class': 'jcs-JobTitle'}).span.get('title')
            comp_name = item.find('span', {'class': 'companyName'}).text
            comp_location = item.find('div',{'class': 'companyLocation'}).text
            
            try:
                description = item.find('div', {'class': 'heading6 tapItem-gutter result-footer'}).div.ul.li.text
            except AttributeError:
                description = item.find('div', {'class': 'heading6 tapItem-gutter result-footer'}).div.text
                
            post_date = item.find('span', {'class': 'date'}).text
            jop_url = 'https://th.indeed.com' + item.find('a', {'class': 'jcs-JobTitle'}).get('href')
        except AttributeError as e:
            print('', end='')
            
        record = (title, comp_name, comp_location, post_date, description, jop_url)
        if record:
            records.append(record)

driver.close()

print(f"\nScraped. total {len(records)+1} records")

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\Pleum\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


Scraping from url https://th.indeed.com/งาน?q=Data+Science&start=1 page 1
Scraping from url https://th.indeed.com/งาน?q=Data+Science&start=2 page 2
Scraping from url https://th.indeed.com/งาน?q=Data+Science&start=3 page 3
Scraping from url https://th.indeed.com/งาน?q=Data+Science&start=4 page 4
Scraping from url https://th.indeed.com/งาน?q=Data+Science&start=5 page 5
Scraping from url https://th.indeed.com/งาน?q=Data+Science&start=6 page 6
Scraping from url https://th.indeed.com/งาน?q=Data+Science&start=7 page 7
Scraping from url https://th.indeed.com/งาน?q=Data+Science&start=8 page 8
Scraping from url https://th.indeed.com/งาน?q=Data+Science&start=9 page 9


In [55]:
# show records
df = pd.DataFrame(records)
df.columns = col_name

df.head()

,JopTitle,CompName,CompLocation,PostDate,Description,JopURL
0,Talent Acquisition Support,PwC,กรุงเทพมหานคร,Posted7 days ago,Our team actively recruits for full time emplo...,https://th.indeed.com/rc/clk?jk=2e2bea387f5ad6...
1,Application Support (Customer Service),Wealth Management System,"คลองสาน, กรุงเทพมหานคร",Posted30+ days ago,Perform problem analysis and provide applicati...,https://th.indeed.com/rc/clk?jk=7918c55a5e4b33...
2,IT support,บริษัทจัดหางาน เวิร์คเมท โซลูชั่นส์ จำกัด,"คลองเตย, กรุงเทพมหานคร",EmployerActive 2 days ago,การแก้ไขปัญหาแอปพลิเคชันและทำงานร่วมกับทีมภายใ...,https://th.indeed.com/company/%E0%B8%9A%E0%B8%...
3,IT One - SAP SD support,Accenture,กรุงเทพมหานคร,Posted6 days ago,Join AM SLM to support SAP SD for SCG and othe...,https://th.indeed.com/rc/clk?jk=28680d79588866...
4,Regional Policy Support Officer,United Nations ESCAP,กรุงเทพมหานคร,Posted8 days ago,Framework; support coordination of migration p...,https://th.indeed.com/rc/clk?jk=524ff63d89b1ed...


In [56]:
# save data to csv file
search_term = search_term.replace(' ', '_')

from datetime import datetime
cur_date = datetime.today().strftime('%d-%m-%Y')

with open(f'{search_term} {cur_date}.csv', 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.writer(f)
    writer.writerow(col_name)
    writer.writerows(records)
    
print('finish')

finish
